# 1. Import Dataset



In [1]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_classif
from scipy.stats import pearsonr
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics

In [2]:
# !pip install -q kaggle
# import os

# os.environ['KAGGLE_USERNAME'] = 'nisasalvia'
# os.environ['KAGGLE_KEY'] = '75a45c7f7de251ea57c14063ec6fb0f9'

# !kaggle datasets download -d uciml/pima-indians-diabetes-database --quiet
# !unzip -q pima-indians-diabetes-database.zip

df = pd.read_csv('diabetes.csv')

In [3]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
db = df.copy()
db

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


# 2. Preprocessing

## Cek Dataset

In [5]:
len(df.columns)

9

In [6]:
pd.options.display.float_format = '{:.2f}'.format
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.00,768.00,768.00,768.00,768.00,768.00,768.00,768.00,768.00
mean,3.85,120.89,69.11,20.54,79.80,31.99,0.47,33.24,0.35
std,3.37,31.97,19.36,15.95,115.24,7.88,0.33,11.76,0.48
min,0.00,0.00,0.00,0.00,0.00,0.00,0.08,21.00,0.00
25%,1.00,99.00,62.00,0.00,0.00,27.30,0.24,24.00,0.00
50%,3.00,117.00,72.00,23.00,30.50,32.00,0.37,29.00,0.00
75%,6.00,140.25,80.00,32.00,127.25,36.60,0.63,41.00,1.00
max,17.00,199.00,122.00,99.00,846.00,67.10,2.42,81.00,1.00


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [8]:
df.rename(columns={'DiabetesPedigreeFunction': 'DPF'}, inplace=True)
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DPF,Age,Outcome
0,6,148,72,35,0,33.60,0.63,50,1
1,1,85,66,29,0,26.60,0.35,31,0
2,8,183,64,0,0,23.30,0.67,32,1
3,1,89,66,23,94,28.10,0.17,21,0
4,0,137,40,35,168,43.10,2.29,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.90,0.17,63,0
764,2,122,70,27,0,36.80,0.34,27,0
765,5,121,72,23,112,26.20,0.24,30,0
766,1,126,60,0,0,30.10,0.35,47,1


## Missing Values & Imputasi

In [9]:
missing_values = df.isnull().sum()
print("Missing values per column:\n", missing_values)

Missing values per column:
 Pregnancies      0
Glucose          0
BloodPressure    0
SkinThickness    0
Insulin          0
BMI              0
DPF              0
Age              0
Outcome          0
dtype: int64


In [10]:
#mengecek nilai 0 pada dataset dan menggantinya ke NaN
df[['Glucose','BloodPressure','SkinThickness','Insulin','BMI']] = df[['Glucose','BloodPressure','SkinThickness','Insulin','BMI']].replace(0,np.nan)
df.isnull().sum()

Pregnancies        0
Glucose            5
BloodPressure     35
SkinThickness    227
Insulin          374
BMI               11
DPF                0
Age                0
Outcome            0
dtype: int64

In [11]:
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DPF,Age,Outcome
count,768.00,763.00,733.00,541.00,394.00,757.00,768.00,768.00,768.00
mean,3.85,121.69,72.41,29.15,155.55,32.46,0.47,33.24,0.35
std,3.37,30.54,12.38,10.48,118.78,6.92,0.33,11.76,0.48
min,0.00,44.00,24.00,7.00,14.00,18.20,0.08,21.00,0.00
25%,1.00,99.00,64.00,22.00,76.25,27.50,0.24,24.00,0.00
50%,3.00,117.00,72.00,29.00,125.00,32.30,0.37,29.00,0.00
75%,6.00,141.00,80.00,36.00,190.00,36.60,0.63,41.00,1.00
max,17.00,199.00,122.00,99.00,846.00,67.10,2.42,81.00,1.00


In [12]:
#melakukan imputasi rata-rata berdasarkan outcome
imp_mean = ['Glucose', 'BloodPressure', 'SkinThickness', 'BMI']
df[imp_mean] = df.groupby('Outcome')[imp_mean].transform(lambda x: x.fillna(x.mean()))
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DPF,Age,Outcome
0,6,148.00,72.00,35.00,NaN,33.60,0.63,50,1
1,1,85.00,66.00,29.00,NaN,26.60,0.35,31,0
2,8,183.00,64.00,33.00,NaN,23.30,0.67,32,1
3,1,89.00,66.00,23.00,94.00,28.10,0.17,21,0
4,0,137.00,40.00,35.00,168.00,43.10,2.29,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101.00,76.00,48.00,180.00,32.90,0.17,63,0
764,2,122.00,70.00,27.00,NaN,36.80,0.34,27,0
765,5,121.00,72.00,23.00,112.00,26.20,0.24,30,0
766,1,126.00,60.00,33.00,NaN,30.10,0.35,47,1


In [13]:
#melakukan imputasi median berdasarkan outcome
imp_med = ['Insulin']
df[imp_med] = df.groupby('Outcome')[imp_med].transform(lambda x: x.fillna(x.median()))
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DPF,Age,Outcome
0,6,148.00,72.00,35.00,169.50,33.60,0.63,50,1
1,1,85.00,66.00,29.00,102.50,26.60,0.35,31,0
2,8,183.00,64.00,33.00,169.50,23.30,0.67,32,1
3,1,89.00,66.00,23.00,94.00,28.10,0.17,21,0
4,0,137.00,40.00,35.00,168.00,43.10,2.29,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101.00,76.00,48.00,180.00,32.90,0.17,63,0
764,2,122.00,70.00,27.00,102.50,36.80,0.34,27,0
765,5,121.00,72.00,23.00,112.00,26.20,0.24,30,0
766,1,126.00,60.00,33.00,169.50,30.10,0.35,47,1


In [14]:
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DPF,Age,Outcome
count,768.00,768.00,768.00,768.00,768.00,768.00,768.00,768.00,768.00
mean,3.85,121.70,72.43,29.25,141.75,32.45,0.47,33.24,0.35
std,3.37,30.46,12.11,8.92,89.10,6.88,0.33,11.76,0.48
min,0.00,44.00,24.00,7.00,14.00,18.20,0.08,21.00,0.00
25%,1.00,99.75,64.00,25.00,102.50,27.50,0.24,24.00,0.00
50%,3.00,117.00,72.00,28.00,102.50,32.05,0.37,29.00,0.00
75%,6.00,141.00,80.00,33.00,169.50,36.60,0.63,41.00,1.00
max,17.00,199.00,122.00,99.00,846.00,67.10,2.42,81.00,1.00


In [15]:
#cek data duplikat
duplicate_count = df.duplicated().sum()
print("Number of duplicate rows:", duplicate_count)

Number of duplicate rows: 0


## Outlier

notes :    
*   max skin thickness menurut jurnal dari analisis suku itu 45 mm, maybe bisa dibuat dari 50 mm
*   untuk bagian insulin paling normal adalah 222, mungkin bisa dibuat sampe 250 batasnya.



In [16]:
df8i = df.copy() #IQR

### IQR

In [17]:
outlier = ['SkinThickness', 'Insulin', 'DPF']

def remove_outliers(dfis, columns, threshold=1.5):
  cleaned = []
  for df in dfis:
      df_clean = df.copy()
      for col in columns:
          Q1 = df_clean[col].quantile(0.25)
          Q3 = df_clean[col].quantile(0.75)
          IQR = Q3 - Q1
          lower_bound = Q1 - threshold * IQR
          upper_bound = Q3 + threshold * IQR
          df_clean = df_clean[(df_clean[col] >= lower_bound) & (df_clean[col] <= upper_bound)]
      cleaned.append(df_clean)
  return cleaned

dfis = [df8i]

df8i = remove_outliers(dfis, outlier, threshold=2.0)  # Meningkatkan threshold dari 1.5 ke 2.0
df8i

[     Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  DPF  \
 0              6   148.00          72.00          35.00   169.50 33.60 0.63   
 1              1    85.00          66.00          29.00   102.50 26.60 0.35   
 2              8   183.00          64.00          33.00   169.50 23.30 0.67   
 3              1    89.00          66.00          23.00    94.00 28.10 0.17   
 5              5   116.00          74.00          27.24   102.50 25.60 0.20   
 ..           ...      ...            ...            ...      ...   ...  ...   
 763           10   101.00          76.00          48.00   180.00 32.90 0.17   
 764            2   122.00          70.00          27.00   102.50 36.80 0.34   
 765            5   121.00          72.00          23.00   112.00 26.20 0.24   
 766            1   126.00          60.00          33.00   169.50 30.10 0.35   
 767            1    93.00          70.00          31.00   102.50 30.40 0.32   
 
      Age  Outcome  
 0     50        

In [18]:
df8i.describe()

AttributeError: 'list' object has no attribute 'describe'

## Normalisasi

MinMax digunakan untuk yang memiliki distribusi normal.
Standarisasi digunakan untuk yang memiliki skewed data.

### IQR

skewed : pregnancies, insulin, DPF, Age

In [ ]:
from sklearn.preprocessing import MinMaxScaler,StandardScaler
mms = MinMaxScaler() # Normalization
ss = StandardScaler() # Standardization

standard = ['Pregnancies', 'Insulin', 'DPF', 'Age']
minmax = ['BloodPressure', 'SkinThickness', 'Glucose', 'BMI']

for df in [df8i]:
    for col in standard:
        df[col] = ss.fit_transform(df[[col]])
    for col in minmax:
        df[col] = mms.fit_transform(df[[col]])

df8i.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DPF,Age,Outcome
0,0.63,0.68,0.49,0.64,0.86,0.31,0.72,1.44,1
1,-0.86,0.27,0.43,0.49,-0.46,0.17,-0.32,-0.19,0
2,1.23,0.90,0.41,0.59,0.86,0.10,0.89,-0.10,1
3,-0.86,0.29,0.43,0.33,-0.63,0.20,-1.02,-1.04,0
5,0.33,0.47,0.51,0.44,-0.46,0.15,-0.89,-0.27,0


## Feature Engineering (ANNOVA)

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

numerical_features = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness',
                      'Insulin', 'BMI', 'DPF', 'Age']
categorical_features = ['Outcome']

def get_anova_scores(df):
    features = df.loc[:, numerical_features]
    target = df.loc[:, categorical_features]
    selector = SelectKBest(score_func=f_classif, k='all')
    fit = selector.fit(features, target)
    scores_df = pd.DataFrame(fit.scores_, index=features.columns, columns=['ANOVA Score'])
    return scores_df.sort_values(by='ANOVA Score', ascending=False)

### 8 Fitur

In [ ]:
f8i = df8i.iloc[:,:8].values
t8i = df8i.iloc[:,8].values

## Sampling Data

### IQR

In [ ]:
import imblearn
from imblearn.over_sampling import SMOTE
from collections import Counter

smote = SMOTE(sampling_strategy='auto', random_state=42)
f8i, t8i = smote.fit_resample(f8i, t8i)
c8i = Counter(t8i)

counter_df = pd.DataFrame({
    't8i': pd.Series(c8i),
}).astype(int)

print(counter_df)

   t8i  t5i  t4i
1  465  465  465
0  465  465  465


## Split Data

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import RocCurveDisplay
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_recall_curve

In [ ]:
# 8 Feature Set
x_train_8i, x_test_8i, y_train_8i, y_test_8i = train_test_split(f8i, t8i, test_size=0.20, stratify=t8i, random_state=2)  # 8I

## Reshape data

In [ ]:
# 8 Features
x_train_8i = x_train_8i.reshape(-1, 8, 1)
x_test_8i = x_test_8i.reshape(-1, 8, 1)

In [ ]:
print(x_train_8i.shape)
print(x_test_8i.shape)

(744, 8, 1)
(186, 8, 1)
(800, 5, 1)
(200, 4, 1)


# Build Model CNN

In [19]:
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam, AdamW
from tensorflow.keras.callbacks import EarlyStopping

ModuleNotFoundError: No module named 'tensorflow.python'

In [ ]:
param_learning_rates = [1e-2, 1e-3, 1e-4]
param_filters = [16, 32, 64]
param_kernel_size = [2, 3]

#untuk 4-5 fitur
param_kernel_size2 = [1, 2]
param_filters2 = [16, 32]

n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

## 8 Fitur

In [ ]:
def build_model(
    input_shape=(8, 1),
    lr=0.001,          # learning rate
    n_filters=32,      # jumlah filter
    kernel_size=3,     # ukuran kernel
    dropout_rate=0.0,  # dropout
):
    model = Sequential()
    model.add(Conv1D(filters=n_filters,
                     kernel_size=kernel_size,
                     activation='relu',
                     input_shape=input_shape))
    model.add(MaxPooling1D(pool_size=2))

    # model.add(Conv1D(filters=n_filters*2, kernel_size=kernel_size, activation='relu'))
    # model.add(MaxPooling1D(pool_size=2))

    model.add(Flatten())

    if dropout_rate > 0:
        model.add(Dropout(dropout_rate))

    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    optimizer = Adam(learning_rate=lr)
    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model

### 8 IQR

In [ ]:
best_config8i = None
best_accuracy8i = 0.0

# Looping semua kombinasi
for lr in param_learning_rates:
    for f in param_filters:
        for k in param_kernel_size:

            cv_accuracies8i = []

           # 4.1 Looping untuk tiap fold
            for train_index_8i, val_index_8i in skf.split(x_train_8i, y_train_8i):
                X_fold_train_8i = x_train_8i[train_index_8i]
                X_fold_val_8i   = x_train_8i[val_index_8i]
                y_fold_train_8i = y_train_8i[train_index_8i]
                y_fold_val_8i   = y_train_8i[val_index_8i]

                # 4.2 Build model
                model_8i = build_model(
                    input_shape=(8, 1),
                    lr=lr,
                    n_filters=f,
                    kernel_size=k,
                    dropout_rate=0.2
                )

                # Callback EarlyStopping
                es = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

                # Training
                history8i = model_8i.fit(
                    x_train_8i, y_train_8i,
                    epochs=100,            #edit
                    batch_size=32,        #edit
                    validation_data=(X_fold_val_8i, y_fold_val_8i),
                    callbacks=[es],
                    verbose=0  # 1 jika ingin menampilkan progres training
                )

                # Evaluasi pada fold ini
                loss8i, accuracy8i = model_8i.evaluate(X_fold_val_8i, y_fold_val_8i, verbose=1)
                cv_accuracies8i.append(accuracy8i)

            # Rata-rata akurasi dari K-Fold
            mean_acc8i = np.mean(cv_accuracies8i)

            # Simpan config terbaik
            if mean_acc8i > best_accuracy8i:
                best_accuracy8i = mean_acc8i
                best_config8i = (lr, f, k)

            print(f"[INFO] LR={lr}, Filters={f}, KernelSize={k}, MeanCVAcc={mean_acc8i:.4f}")

print("========================================================")
print(f"Best Config: LR={best_config8i[0]}, Filters={best_config8i[1]}, Kernel={best_config8i[2]}")
print(f"Best Mean CV Accuracy: {best_accuracy8i:.4f}")
print("========================================================")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9230 - loss: 0.2478
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8939 - loss: 0.2463
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.8995 - loss: 0.2384
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9051 - loss: 0.2856
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8898 - loss: 0.1971
[INFO] LR=0.01, Filters=16, KernelSize=2, MeanCVAcc=0.8925


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9048 - loss: 0.2320
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.8919 - loss: 0.2159
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9070 - loss: 0.2020
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.9109 - loss: 0.2928
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9140 - loss: 0.1592
[INFO] LR=0.01, Filters=16, KernelSize=3, MeanCVAcc=0.9059


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.9029 - loss: 0.2760
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8666 - loss: 0.3582
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8549 - loss: 0.3582
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9051 - loss: 0.3380
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.8916 - loss: 0.3202
[INFO] LR=0.0001, Filters=32, KernelSize=3, MeanCVAcc=0.8777


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8805 - loss: 0.2784
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8287 - loss: 0.3402
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8418 - loss: 0.3455
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8920 - loss: 0.3464
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8916 - loss: 0.2939
[INFO] LR=0.0001, Filters=64, KernelSize=2, MeanCVAcc=0.8656


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8968 - loss: 0.2658 
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8642 - loss: 0.3272
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8680 - loss: 0.3458
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9073 - loss: 0.3212
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8951 - loss: 0.3034
[INFO] LR=0.0001, Filters=64, KernelSize=3, MeanCVAcc=0.8831
Best Config: LR=0.01, Filters=32, Kernel=2
Best Mean CV Accuracy: 0.9247


In [ ]:
best_lr, best_filters, best_k = best_config8i

model_final_8i = build_model(
    input_shape=(8, 1),
    lr=best_lr,
    n_filters=best_filters,
    kernel_size=best_k,
    dropout_rate=0.2
)

es_final_8i = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Latih dengan semua data train (atau pakai validation_split, atau siapkan data val lain)
history_final_8i = model_final_8i.fit(
    x_train_8i, y_train_8i,
    epochs=100,
    batch_size=32,
    validation_split=0.2,
    callbacks=[es_final_8i],
    verbose=1
)

Epoch 1/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - accuracy: 0.7164 - loss: 0.5360 - val_accuracy: 0.8255 - val_loss: 0.5357
Epoch 2/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8555 - loss: 0.4308 - val_accuracy: 0.8389 - val_loss: 0.4265
Epoch 3/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8894 - loss: 0.3275 - val_accuracy: 0.8322 - val_loss: 0.4176
Epoch 4/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8715 - loss: 0.3118 - val_accuracy: 0.8121 - val_loss: 0.4230
Epoch 5/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8618 - loss: 0.3231 - val_accuracy: 0.8188 - val_loss: 0.4397
Epoch 6/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8856 - loss: 0.3170 - val_accuracy: 0.8054 - val_loss: 0.4211
Epoch 7/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8898 - loss: 0.2945 - val_accuracy: 0.8389 - val_loss: 0.4102
Epoch 8/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8849 - loss: 0.2949 - val_accuracy: 0.7987 

In [ ]:
test_loss8i, test_acc8i = model_final_8i.evaluate(x_test_8i, y_test_8i, verbose=0)
print(f"Test Accuracy: {test_acc8i:.4f}")

Test Accuracy: 0.8817


In [ ]:
import pickle
with open("summary_8i.pkl", "wb") as f:
    pickle.dump({
        "best_accuracy": best_accuracy8i,
        "test_accuracy": test_acc8i,
        "best_config":   best_config8i
    }, f)

In [ ]:
print("\n--- Final 8 Feature model architecture ---")
model_final_8i.summary()


--- Final 8 Feature model architecture ---


Model: "sequential_90"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_90 (Conv1D)              │ (None, 7, 32)          │            96 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_90 (MaxPooling1D) │ (None, 3, 32)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_90 (Flatten)            │ (None, 96)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_90 (Dropout)            │ (None, 96)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_180 (Dense)               │ (None, 64)             │         6,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_181 (Dense)               │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,109 (74.65 KB)

 Trainable params: 6,369 (24.88 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 12,740 (49.77 KB)